In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
train_path= pd.read_csv('IA1_train.csv')
val_path= pd.read_csv('IA1_dev.csv')
val_path

: 

In [ ]:
#remove the ID column from both training and validation data
train_without_id = train_path.drop(columns=['id'])
train_without_id.dtypes
test_without_id = val_path.drop(columns=['id'])
test_without_id.dtypes

: 

In [ ]:
#change date into 3 numerical features day, month and year
train_without_id['date']=pd.to_datetime(train_without_id['date'], format='%m/%d/%Y')
#extract month, day and year into separate columns
train_without_id['SaleMonth']= train_without_id['date'].dt.month
train_without_id['SaleDat']= train_without_id['date'].dt.day
train_without_id['SaleYear']= train_without_id['date'].dt.year
#drop the original date column
train_without_id=train_without_id.drop(columns=['date'])
train_without_id.dtypes



: 

In [ ]:
test_without_id['date']=pd.to_datetime(test_without_id['date'], format='%m/%d/%Y')
#extract month, day and year into separate columns
test_without_id['SaleMonth']= test_without_id['date'].dt.month
test_without_id['SaleDat']= test_without_id['date'].dt.day
test_without_id['SaleYear']= test_without_id['date'].dt.year
#drop the original date column
test_without_id=test_without_id.drop(columns=['date'])
test_without_id.dtypes

: 

In [ ]:
#The feature yr_renovated is set to 0 if the house has not been renovated
train_without_id['age_since_renovated']=np.where(train_without_id['yr_renovated']!=0, 2024-train_without_id['yr_renovated'], 2024-train_without_id['yr_built'])
train_without_id
test_without_id['age_since_renovated']=np.where(test_without_id['yr_renovated']!=0, 2024-test_without_id['yr_renovated'], 2024-test_without_id['yr_built'])
test_without_id
#unique_bedrooms=sorted(train_without_id['age_since_renovated'].unique())
#unique_bedrooms


: 

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
y_train=train_without_id['price']
train_without_price=train_without_id.drop(columns=['price','yr_renovated'])
scaler.fit(train_without_price)
scaler_train=scaler.transform(train_without_price)
len(scaler_train[0])

: 

In [ ]:

y_test=test_without_id['price']
test_without_price=test_without_id.drop(columns=['price','yr_renovated'])
scaler.fit(test_without_price)
scaler_test=scaler.transform(test_without_price)
len(scaler_test[0])


: 

In [ ]:
#Part 1 (15 pts) Generate closed-form solution for reference.
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression()
lr_model.fit(scaler_train, y_train)
y_pred = lr_model.predict(scaler_test)

print("Prediction on training set:", y_pred)
#print("Accuracy on training set:", lr_model.score(X, y))
print(y_test)

from sklearn.metrics import mean_squared_error
rmse_test=mean_squared_error(y_test,y_pred)
print(rmse_train)


: 

In [ ]:
y_train_pred=lr_model.predict(scaler_train)

rmse_train = mean_squared_error(y_train, y_train_pred)
rmse_train

: 

In [ ]:
lr_model.coef_


: 

In [ ]:
lr_model.intercept_

: 

In [ ]:
print(test_without_id.columns.tolist())

: 

In [ ]:
#Part 2 (40 pts) Implement and experiment with batch gradient descent
from sklearn.linear_model import SGDClassifier
lr_model_2 = SGDRegressor(loss='squared_error', alpha=1, n_iter_no_change=3000, verbose=1)
lr_model_2.fit(scaler_train, y_train)
y_pred_2 = lr_model.predict(scaler_test)

print("Prediction on training set:", y_pred_2)
#print("Accuracy on training set:", lr_model.score(X, y))
print(y_test)

from sklearn.metrics import mean_squared_error
rmse_test_2=mean_squared_error(y_test,y_pred_2)
print(rmse_test_2)
y_train_pred_2=lr_model.predict(scaler_train)

rmse_train_2 = mean_squared_error(y_train, y_train_pred_2)
rmse_train

import sys
import io
import matplotlib.pyplot as plt

class DisplayLossCurve(object):
  def __init__(self, print_loss=False):
    self.print_loss = print_loss

  """Make sure the model verbose is set to 1"""
  def __enter__(self):
    self.old_stdout = sys.stdout
    sys.stdout = self.mystdout = io.StringIO()
  
  def __exit__(self, *args, **kwargs):
    sys.stdout = self.old_stdout
    loss_history = self.mystdout.getvalue()
    loss_list = []
    for line in loss_history.split('\n'):
      if(len(line.split("loss: ")) == 1):
        continue
      loss_list.append(float(line.split("loss: ")[-1]))
    plt.figure()
    plt.plot(np.arange(len(loss_list)), loss_list)
    plt.xlabel("Epoch")
    plt.ylabel("Loss")

    if self.print_loss:
      print("=============== Loss Array ===============")
      print(np.array(loss_list))
      
    return True

with DisplayLossCurve(print_loss=True):
    lr_model_2.fit(scaler_train, y_train)



: 

In [ ]:
lr_model_3 = SGDRegressor(loss='squared_error', alpha=0.0001, n_iter_no_change=3000, verbose=1)
with DisplayLossCurve(print_loss=True):
    lr_model_3.fit(scaler_train, y_train)

: 